# <center> **502 Project** </center>
### <center> Team member: Chenxi Liu, Nuo Tian, Mengyu Yu, Yuan Liu </center>

### Data Selection: 
#### Yahoo News Data Set (Part 1 and 2of 35) 




### STEP 1 : Data Extracting and Processing 

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder \
    .appName("project")\
    .master("local[4]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.4")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[4] appName=project>

#### 1. Build Schema Using the README Instruction of the Data Set 
#### 

In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("article_type", StringType(), True),
    StructField("np1", StringType(), True),
    StructField("np2", StringType(), True),
    StructField("context", StringType(), True),
    StructField("source", StringType(), True),
    StructField("category", StringType(), True),
    StructField("location", StringType(), True),
    StructField("time", StringType(), True),])

#### 2. Readin the Data Set 
#### 

In [3]:
df = spark.read.csv("s3://anly502project/data/part-r-00000",sep = "\t",header=False,schema=schema)
df_2 = spark.read.csv("s3://anly502project/data/part-r-00001",sep = "\t",header=False,schema=schema)

#### 3. Verify the Schema 
#### 

In [4]:
#### Data Schema
df.printSchema()
df_2.printSchema()

root
 |-- article_type: string (nullable = true)
 |-- np1: string (nullable = true)
 |-- np2: string (nullable = true)
 |-- context: string (nullable = true)
 |-- source: string (nullable = true)
 |-- category: string (nullable = true)
 |-- location: string (nullable = true)
 |-- time: string (nullable = true)

root
 |-- article_type: string (nullable = true)
 |-- np1: string (nullable = true)
 |-- np2: string (nullable = true)
 |-- context: string (nullable = true)
 |-- source: string (nullable = true)
 |-- category: string (nullable = true)
 |-- location: string (nullable = true)
 |-- time: string (nullable = true)



#### 4. Combine the Data Set
#### 

In [5]:
import functools 

def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 

In [6]:
unioned_df = unionAll([df, df_2])


In [7]:
#### show combined
unioned_df.show(10)

+------------+---------------+---+--------------------+-------------+--------------------+--------------------+-----+
|article_type|            np1|np2|             context|       source|            category|            location| time|
+------------+---------------+---+--------------------+-------------+--------------------+--------------------+-----+
|     article|    Dark Knight|  E|  arg1 and Wall arg2|             |intlnews topstor ...|      , kerala india|14299|
|     article|    Carotenoids|  E|arg1 and caroteno...|             |topstor,health,sc...|                   ,|14660|
|     article|    Communities|  E|arg1 mobilised in...|             |    politics topstor|                   ,|14026|
|     article|    Carotenoids|  E|arg1 and caroteno...|             |topstor,health,sc...|                   ,|14660|
|            |     Coast bias|  E|arg2 is for East ...|             |      sports topstor| columbus, ohio u...|13956|
|     article|Commerce office|  E|arg1 at DDD Linco...| 

In [8]:
#### show number of the rows 
unioned_df.count()

196017242

In [9]:
####Drop unrelated Column
unioned_df = unioned_df.drop("source").drop("article_type").drop("location").drop("time")

In [10]:
####Show data frame after dropping unrelated columns 
unioned_df.show(10)

+---------------+---+--------------------+--------------------+
|            np1|np2|             context|            category|
+---------------+---+--------------------+--------------------+
|    Dark Knight|  E|  arg1 and Wall arg2|intlnews topstor ...|
|    Carotenoids|  E|arg1 and caroteno...|topstor,health,sc...|
|    Communities|  E|arg1 mobilised in...|    politics topstor|
|    Carotenoids|  E|arg1 and caroteno...|topstor,health,sc...|
|     Coast bias|  E|arg2 is for East ...|      sports topstor|
|Commerce office|  E|arg1 at DDD Linco...|topstor,lifestle,...|
| 75-minute mark|  E|arg1 in a pulsati...|      topstor,sports|
| Brigham Circle|  E|      arg2 past arg1|             topstor|
| Brigham Circle|  E| arg2 trains at arg1|localnews headlin...|
| Brigham Circle|  E|arg2 line service...|             topstor|
+---------------+---+--------------------+--------------------+
only showing top 10 rows



#### 5. Select the very top category by using multiple split 
#### 

In [11]:
from pyspark.sql.functions import split
split_col = split(unioned_df['category'], ',')
unioned_df = unioned_df.withColumn('category', split_col.getItem(0))

In [12]:
unioned_df.show(10)

+---------------+---+--------------------+--------------------+
|            np1|np2|             context|            category|
+---------------+---+--------------------+--------------------+
|    Dark Knight|  E|  arg1 and Wall arg2|intlnews topstor ...|
|    Carotenoids|  E|arg1 and caroteno...|             topstor|
|    Communities|  E|arg1 mobilised in...|    politics topstor|
|    Carotenoids|  E|arg1 and caroteno...|             topstor|
|     Coast bias|  E|arg2 is for East ...|      sports topstor|
|Commerce office|  E|arg1 at DDD Linco...|             topstor|
| 75-minute mark|  E|arg1 in a pulsati...|             topstor|
| Brigham Circle|  E|      arg2 past arg1|             topstor|
| Brigham Circle|  E| arg2 trains at arg1|localnews headlin...|
| Brigham Circle|  E|arg2 line service...|             topstor|
+---------------+---+--------------------+--------------------+
only showing top 10 rows



In [13]:
split_col_2 = split(unioned_df['category'], ' ')
unioned_df = unioned_df.withColumn('category', split_col_2.getItem(0))

In [14]:
split_col_3 = split(unioned_df['category'], '_')
unioned_df = unioned_df.withColumn('category', split_col_3.getItem(0))

In [15]:
split_col_4 = split(unioned_df['category'], '-')
unioned_df = unioned_df.withColumn('category', split_col_4.getItem(0))

In [16]:
#### Show data frame after filtering the category
unioned_df.show(10)

+---------------+---+--------------------+---------+
|            np1|np2|             context| category|
+---------------+---+--------------------+---------+
|    Dark Knight|  E|  arg1 and Wall arg2| intlnews|
|    Carotenoids|  E|arg1 and caroteno...|  topstor|
|    Communities|  E|arg1 mobilised in...| politics|
|    Carotenoids|  E|arg1 and caroteno...|  topstor|
|     Coast bias|  E|arg2 is for East ...|   sports|
|Commerce office|  E|arg1 at DDD Linco...|  topstor|
| 75-minute mark|  E|arg1 in a pulsati...|  topstor|
| Brigham Circle|  E|      arg2 past arg1|  topstor|
| Brigham Circle|  E| arg2 trains at arg1|localnews|
| Brigham Circle|  E|arg2 line service...|  topstor|
+---------------+---+--------------------+---------+
only showing top 10 rows



#### 6. Category (label) insight 
#### 

In [ ]:
#### Get some insight of the label 
unioned_df.groupBy("category").count().show()

In [ ]:
#### Filter out the null and empty category 
unioned_df = unioned_df.filter(unioned_df.category.isNotNull())
unioned_df = unioned_df.filter(unioned_df.category != '')

In [ ]:
#### More insight 
from pyspark.sql.functions import countDistinct
unioned_df.agg(countDistinct(unioned_df.category).alias('distinct_label')).collect()
##### We can see we have around 1116 distinct labels which some of it does not make any sense so we dicided to filter it out 

#### 7. Using the SQL to filter the category (label)
#### 

In [ ]:
import pyspark.sql as sql
count_df = unioned_df.groupBy("category").count()
count_df.createOrReplaceTempView("count_df")
count_rank_df = spark.sql("SELECT category, count FROM count_df ORDER BY count DESC LIMIT 15")

In [ ]:
count_rank_df.show(15)

In [ ]:
unioned_df.createOrReplaceTempView("unioned_df")
count_rank_df.createOrReplaceTempView("count_rank_df")
df_final = spark.sql("SELECT * FROM unioned_df WHERE unioned_df.category IN (SELECT category FROM count_rank_df)")

In [ ]:
df_final.show(10)

In [ ]:
df_final.agg(countDistinct(df_final.category).alias('distinct_label')).collect()

In [ ]:
#### Verfied the final data frame 
df_final.groupBy("category").count().show()

In [20]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer, 
                                LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline

In [21]:
import nltk
nltk.download('stopwords')
nltk.download('words')

from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')
eng_stopwords.append('xxxx')

[nltk_data] Downloading package stopwords to /home/hadoop/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/hadoop/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [22]:
documentAssembler = DocumentAssembler() \
    .setInputCol('context') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# note normalizer defaults to changing all words to lowercase.
# Use .setLowercase(False) to maintain input case.
normalizer = Normalizer() \
    .setInputCols(['token']) \
    .setOutputCol('normalized') \
    .setLowercase(True)

# note that lemmatizer needs a dictionary. So I used the pre-trained
# model (note that it defaults to english)
lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(['normalized']) \
    .setOutputCol('lemma') \

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(['lemma']) \
    .setOutputCol('clean_lemma') \
    .setCaseSensitive(False) \
    .setStopWords(eng_stopwords)

# finisher converts tokens to human-readable output
finisher = Finisher() \
    .setInputCols(['clean_lemma']) \
    .setCleanAnnotations(False)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [7]:
import sparknlp
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

spark = sparknlp.start()
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.4.5
Apache Spark version:  2.4.5


In [52]:
bert = BertEmbeddings.pretrained('bert_base_cased', 'en') \
    .setInputCols(["sentence",'token'])\
    .setOutputCol("bert")\
    .setCaseSensitive(False)\
    .setPoolingLayer(0) # default 0

bert_base_cased download started this may take some time.


TypeError: 'JavaPackage' object is not callable

In [12]:
sc.stop()

In [13]:
SparkSession._instantiatedContext = None